# How to compute the surface brightness profile for a dust scattering halo

The `newdust.halos` module can compute the surface brightness profile for a dust scattering halo under a variety of geometric assumptions. This functionality is provided by two main subclasses of the `newdust.halos.Halo` superclass:

* `newdust.halos.UniformGalHalo` assumes that the dust is distributed uniformly between the telescope and the X-ray point source

* `newdust.halos.ScreenGalHalo` assumes that the dust is located at an infinitesimally thin screen at the location $x = (1 - d/D)$ where $d$ is the distance between the telescope and the screen, and $D$ is the distance between the telescope and the X-ray point source

Through inheritance, all halos have the same attributes as those defined in the `newdust.halos.Halo` class. This notebook demonstrates the functionality of a subset of these attributes:

* A `lam` array describing the energy or wavelength grid for the halo

* A `theta` array describing the angular grid for the halo

* A `calculate` method that takes a `newdust.GrainPop` object and any other arguments necessary to complete the calculation (such as $x$, for a ScreenGalHalo). The results are stored in the `norm_int` and `taux` arrays.

* A 2D `norm_int` array of shape `(len(lam), len(theta))` that describes the normalized intensity (intensity divided by flux) for the dust scattring halos, as a function of energy or wavelength

* A `taux` array describing the X-ray scattering opacity as a function of energy or wavelength

* A `calculate_intensity` method that takes the flux spectrum of the X-ray point source as input. It multiplies the flux values by the `norm_int` array and sums them, storing the final results in the `intensity`, `fabs`, and `fhalo` attributes.

* A 1D `intensity` array describing the dust scattering halo surface brightness (e.g., photon/s/cm^2/arcsec^2) integrated over the entire energy/wavelength range

* The X-ray point source flux spectrum for the intensity calculation is stored in the `fabs` array. There are some subtleties around the way this is computed and stored based on the input flux spectrum for `calculate_intensity`, but don't worry about it for now.

* The `fhalo` array describes the total flux contained in the scattering halo as a function of energy or wavelength.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
import astropy.constants as c

import newdust

%matplotlib inline

## Initial calculation of a dust scattering halo model

To initialize the dust scattering halo object, we need to input a grid of energies and angles (representing the angular distances over which the dust scattring halo is visible).

In [ ]:
NE, NTH = 50, 200
EVALS   = np.logspace(-1, 1, NE) * u.keV
THVALS  = np.logspace(-1, 4, NTH) * u.arcsec

# Initialize each halo type
uniform_halo = newdust.halos.UniformGalHalo(EVALS, THVALS)
screen_halo = newdust.halos.ScreenGalHalo(EVALS, THVALS)

To calculate the intensity profile, we need to tell it what types of dust grains are doing the scattering (grain size distribution and composition). This information is stored in a `newdust.GrainPop` object. 

In the case below, we use the Rayleigh-Gans plus Drude approximation, which treats the dust grain as a ball of electrons, making it relatively insensitive to composition, compared to Mie scattering. The shortcut function, `newdust.make_MRN_RGDrude` sets up a standard power-law distribution of dust grain sizes (see MRN 1977) with the RG-Drude approximation for composition.

In [ ]:
GPOP = newdust.make_MRN_RGDrude(md=1.e-6)

Now we use the `calculate` method to run the scattering halo calculation with the specified dust grain population.

Here, I'm using $x=0.33$ which means the dust screen is 2/3 of the way along the sight line between the telescope and the background X-ray point source.

The `%%time` command at the top of the cell makes it print the amount of time it took to run the calculation.

In [ ]:
%%time
screen_halo.calculate(GPOP, x=0.33)

For the case of dust distributed uniformly along the line of sight, it performs the integral over the scattering profile from a large number of `x` values. We can specify how fine of a grid to integrate over using the `nx` keyword. Try adjusting the `nx` value to see how the speed of this calculation changes. You should also note how changing the `nx` value affects the shape of the scattering profile when we plot it, later on in this notebook.

In [ ]:
%%time
uniform_halo.calculate(GPOP, nx=100)

The results of the calculation are stored in the `norm_int` attribute, which has a shape of `(len(lam), len(theta))`. This gives the normalized intensity (surface brightness divided by flux) for the dust scattering halo, as a function of energy. To view a single halo profile, we have to select what energy to plot.

Below, I choose an energy index value at random to plot. Try other index values to see how the scattering halos profile shapes change with energy.

In [ ]:
i = 10

plt.plot(THVALS, screen_halo.norm_int[i,:], label='Screen halo')
plt.plot(THVALS, uniform_halo.norm_int[i,:], label='Uniform halo')
plt.loglog()
plt.legend()

plt.title("Photon Energy = {:.2f}".format(EVALS[i]))
plt.xlabel(r"Angular Distance from Point Source ({})".format(THVALS.unit))
plt.ylabel(r"Normalized Intensity ({})".format(screen_halo.norm_int.unit))

## Testing that the halo calculation is correct

Here we want to test that the scattring halo calculation has proceded correctly. Using the trapezoidal integration function from numpy, we can integrate the scattering halo surface brightness profile described in `norm_int`, in just a few lines.

In [ ]:
# Make a 2D grid of theta values to match the shape 
# of the `norm_int` array
theta_grid = np.repeat(THVALS.reshape(1, NTH), NE, axis=0)

# Integrate the surface brightness profile.
# Don't forget the factor of 2 pi sin(theta), for spherical coordinates
# Because we are using small angles, we can use sin(theta) -> theta
# Also because the units of norm_int are arcsec^-2, we can keep theta_grid
# in units of arcsec
integrated_uniform_halo = np.trapz(uniform_halo.norm_int * \
                                    2.0 * np.pi * theta_grid, 
                                    THVALS, axis=1)
integrated_screen_halo  = np.trapz(screen_halo.norm_int * \
                                    2.0 * np.pi * theta_grid, 
                                    THVALS, axis=1)

The plots below demonstrate that integrating over the normalized dust scattering halos returns the scattering optical depth for that energy value. This is correct for the optically thin case ($\tau_{\rm sca} < 1$).

Note that both scattering halos (Uniform and Screen) have the same `taux` values -- only different shapes!

In [ ]:
plt.plot(EVALS, uniform_halo.taux, 'rs', label='Uniform')
plt.plot(EVALS, integrated_uniform_halo, color='k', label='')

plt.plot(EVALS, screen_halo.taux, 'bo', label='Screen')
plt.plot(EVALS, integrated_screen_halo, color='k', label='')

plt.loglog()
plt.legend()

plt.xlabel(EVALS.unit)
plt.ylabel(r'X-ray Scattering Opacity ($\tau_{\rm sca}$)')

This plot shows how accurate our calculation was by plotting the fractional difference between our integrated halos and the `taux` value. A fractional difference of 0 means that our integration exactly matches the `taux` value. In general, our calculations are tuned to a few percent accuracty. Later on, we may wish to evaluate ways to obtain higher accuracy.

In [ ]:
plt.plot(EVALS, integrated_uniform_halo/uniform_halo.taux-1.0, 
         label='Uniform')
plt.plot(EVALS, integrated_screen_halo/screen_halo.taux-1.0, 
         label='Screen')
plt.legend()
plt.xlabel(EVALS.unit)
plt.ylabel('Fractional Difference (halo integration/taux - 1.0)')

## Calculate the scattering halo intensity, given a point source spectrum

Recall that the `newdust.Halo.calculate_intensity` method will calculate the total dust scattering halo intensity, summed across the energy or wavelength grid specified in the `newdust.Halo.lam` attribute.

Below, I construct a model spectrum that is similar in shape to many Galactic X-ray binaries. The `calculate_intensity` code uses `numpy.sum` to integrate. For this type of calculation to be accurate, the input flux spectrum must represent the "bin-integrated" flux, meaning that each point in the spectrum represents the total number of counts (or energy) within some energy/wavelength bin where the `lam` array represents the center of those energy/wavelength bins.

The bin-integrated flux has units, e.g., photon counts/cm$^2$/s (rather than the specific flux, which has units of, e.g., photons/cm$^2$/s/keV). Note how I have assigned these units to the model spectrum.

In [ ]:
# Approximation of a typical Galactic X-ray Binary spectrum
FABS = 1.0 * np.power(EVALS.value, -2.5) * \
       np.exp(-0.1 * np.power(EVALS.value, -3.5)) * \
       u.Unit('ct s^-1 cm^-2')
plt.plot(EVALS, FABS)
plt.loglog()
plt.xlabel(EVALS.unit)
plt.ylabel(FABS.unit)
plt.ylim(1.e-12, 1.e3)

Now we can run the `calculate_intensity` method for each model halo, with the model X-ray binary spectrum as input.

In [ ]:
screen_halo.calculate_intensity(FABS)
uniform_halo.calculate_intensity(FABS)

The results of the calculation are stored in the `intensity` attribute of each Halo. The results represent the sum of the surface brightness profiles at all energies, resulting in a 1D array with length equal to `len(theta)`.

Below, we plot the scattering halo surface brightness profile, as it is stored in the `intensity` attribute. Note how the units have carried through from the model spectrum and the `norm_int` array.

In [ ]:
plt.plot(THVALS, screen_halo.intensity, label='Screen halo')
plt.plot(THVALS, uniform_halo.intensity, label='Uniform halo')
plt.loglog()
plt.legend()

plt.xlabel(r"Angular Distance from Point Source ({})".format(THVALS.unit))
plt.ylabel(r"{}".format(screen_halo.intensity.unit))
plt.title("Scattering halo surface brightness ({:.2f} - {:.2f} keV)".format(
          EVALS[0].to('keV').value, EVALS[-1].to('keV').value))

If we want to know the total flux in the scattering halo, we need to integrate it over the total area. Don't forget the factor of $2 \pi \theta$

Note also, because both halos were calculated with the same GrainPop and dust mass column, they integrate to the same values, despite having different shapes.

In [ ]:
print("Total flux in Uniform scattering halo {:.3f}".format( 
      np.trapz(uniform_halo.intensity * 2.0 * np.pi * THVALS, THVALS)))

print("Total flux in Screen scattering halo {:.3f}".format( 
      np.trapz(uniform_halo.intensity * 2.0 * np.pi * THVALS, THVALS)))

After running `calculate_intensity`, the `fhalo` attribute will return the flux spectrum of the scattering halo as a function of energy/wavelength.

In [ ]:
plt.plot(EVALS, FABS, color='k', label='Input Flux for calculate_intensity')
plt.plot(EVALS, FABS, color='r', label='ISM Absorbed Flux')
plt.plot(EVALS, uniform_halo.fhalo, 
         color='g', label='Uniform Halo Flux')
plt.plot(EVALS, screen_halo.fhalo, 
         color='b', ls='--', label='Screen Halo Flux')
plt.loglog()
plt.legend()
plt.ylim(1.e-12, 1e3)
plt.xlabel(EVALS.unit)
plt.ylabel('Flux ({})'.format(uniform_halo.fhalo.unit))

## Indexing and slicing dust scattering halo arrays

Just like a typicaly numpy array, we can index or slice the dust scattering halos. Instead of indexing with natural numbers corresponding to the position in th array, we can slice the Halo objects according to energy or wavelength. Slicing returns a new `newdust.Halo` object corresponding to a sub-set of the original Halo.

To index or slice, we must use numbers that correspond to the same units used to define the `newdust.Halo.lam` array. So if you defined `lam` using Angstrom units, you cannot specify a slice using keV units.

In [ ]:
uh1 = uniform_halo[:3.0] # Halo object covering a subset of energies: Everything below 3 keV
uh2 = uniform_halo[3.0:] # Halo object covering a subset of energies: Everything above 3 keV

# Verify the energy ranges
print("Uniform Halo 1: {:.2f} - {:.2f} keV".format(uh1.lam[0].value, uh1.lam[-1].value))
print("Uniform Halo 2: {:.2f} - {:.2f} keV".format(uh2.lam[0].value, uh2.lam[-1].value))

All of the previous calculations carry over to the sliced Halo object. Including the intensity calculations!

In [ ]:
# Plot the taux values
plt.plot(uh1.lam, uh1.taux, 'rs', label='Uniform Halo Slice 1')
plt.plot(uh2.lam, uh2.taux, 'gs', label='Uniform Halo Slice 2')
plt.plot(uniform_halo.lam, uniform_halo.taux, 'k-', label='Original Uniform Halo')
plt.loglog()
plt.xlabel(uh1.lam.unit)
plt.ylabel('X-ray scattering opacity (taux)')
plt.legend()

In [ ]:
# Plot the scattering halo intensities for each energy range
plt.plot(uh1.theta, uh1.intensity, 
         label='{:.2f}-{:.2f} keV'.format(uh1.lam[0].value, uh1.lam[-1].value))
plt.plot(uh2.theta, uh2.intensity, 
         label='{:.2f}-{:.2f} keV'.format(uh2.lam[0].value, uh2.lam[-1].value))
plt.loglog()
plt.legend()
plt.xlabel(uh1.theta.unit)
plt.ylabel(uh2.intensity.unit)
plt.title("Uniform Halo surface brightness profile")